In [4]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matrix_method as matrix_method

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Short Description of the Method
The canonical definition of Mutual Information is 
$$ 
I(X;Y) = - \sum_{x,y} P_{X,Y}(x,y) \log \frac{P_{X,Y}(x,y)}{P_X(x)P_Y(y)}
$$
where $P_{X,Y}$ is the joint distribution, and $P_X, P_Y$ are the marginals.

But mutual information has many equivalent definitions involving the entropy. When using Von Neumann entropy, the most convenient is
$$
I(X,Y) = H(X) + H(Y) - H(X,Y)
$$
where $H(X)$ is the entropy of $X$, and $H(X,Y)$ is the joint entropy of $X$ and $Y$.

Von Neumann entropy was defined over the density matrix $\rho$ of a quantum state. It works by casting the matrix into the spectral domain and computing the entropy of its eigenvalues. 
$$ S(X) = - \sum_i \eta_i \log \eta_i $$
@Passerini2021 describes a map between the density matrix and the normalized graph laplacian. Specifically, the Laplacian $L = D - A$ is computed, and divided by the (scalar) trace of $D$.

So, with the background behind us -

Given two matrices X and Y (which can contain different numbers of points, but currently must contain points in the same dimension):
1. Computes a graph from X and Y, and `concat(X,Y)`.
2. Gets the normalized laplacian of each graph
3. Calculates the entropy from each graph, and
4. Returns the mutual information $I(x,y) = H(X) + H(Y) - H(X,Y)$.

Here's an example computation between two random matrices. This uses the graphtools backend to compute a knn graph, with k=10.

In [5]:
np.random.seed(42)

In [11]:
X = np.random.rand(40,5)
Y = np.random.rand(40,5)
a = matrix_method.spectral_mutual_information(X,Y)
print(f"Mutual info {a}")

A matrix from sklearn [[0. 0. 0. ... 0. 1. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 1. 0. 1.]
 ...
 [0. 0. 1. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]
total degree sum 200.0
Rho matrix [[ 0.025  0.     0.    ...  0.    -0.005 -0.005]
 [ 0.     0.025  0.    ...  0.     0.    -0.005]
 [ 0.     0.     0.025 ... -0.005  0.    -0.005]
 ...
 [ 0.     0.    -0.005 ...  0.025  0.     0.   ]
 [-0.005  0.    -0.005 ...  0.     0.025  0.   ]
 [ 0.     0.    -0.005 ...  0.     0.     0.025]]
evals [0.00201904 0.00270899 0.00444318 0.00857495 0.00958492 0.01118882
 0.01221174 0.01229411 0.01696092 0.01741246 0.01780107 0.01907709
 0.02043197 0.02251095 0.02328043 0.02384419 0.02403422 0.02465926
 0.02639737 0.02681432 0.02773603 0.02801271 0.02906734 0.02941753
 0.03044031 0.03139203 0.03192282 0.03255479 0.03304776 0.03320132
 0.03360654 0.03435386 0.0349876  0.0358253  0.03655945 0.03742659
 0.03797023 0.0390694  0.03983098 0.04136548]
A matrix from sklearn [[0. 0. 0. .

As a sanity check, the mutual information should be much higher when $X = Y$.

In [10]:
a = matrix_method.spectral_mutual_information(X,X)
print(f"Mutual info {a}")

A matrix from sklearn [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
total degree sum 200.0
Rho matrix [[ 0.025  0.     0.    ...  0.     0.     0.   ]
 [ 0.     0.025  0.    ...  0.     0.     0.   ]
 [ 0.     0.     0.025 ...  0.    -0.005  0.   ]
 ...
 [ 0.     0.     0.    ...  0.025  0.     0.   ]
 [ 0.     0.     0.    ...  0.     0.025  0.   ]
 [ 0.     0.     0.    ...  0.     0.     0.025]]
evals [0.00404828 0.00020616 0.00321572 0.00573291 0.00749887 0.01315717
 0.01350815 0.01532833 0.0168744  0.01780947 0.0183872  0.02035297
 0.02106554 0.02195156 0.02431966 0.02477468 0.025      0.02551626
 0.02624371 0.02762933 0.02843333 0.02890681 0.03       0.03
 0.03020142 0.03102244 0.03186361 0.03219402 0.03229119 0.03308293
 0.03396032 0.03463309 0.03482498 0.03579037 0.03633596 0.03695779
 0.0378361  0.0382874  0.03892843 0.04033831]
A matrix from sklearn [[0. 0. 0. ... 0. 

It doesn't do *much* better.

This seems to be an artifact of the KNN graph construction. In the case where X=Y, each point should have an additional neighbor with zero distance.
Perhaps computation through the affinity matrix does better.

In [64]:
a = matrix_method.spectral_mutual_information_via_affinity(X,Y)
print(f"Mutual info {a}")

adjacency [[0.         0.0894966  0.16786662 ... 0.22603731 0.26477877 0.1659443 ]
 [0.0894966  0.         0.2567415  ... 0.09208004 0.39990861 0.41927736]
 [0.16786662 0.2567415  0.         ... 0.05018176 0.28066441 0.8046731 ]
 ...
 [0.22603731 0.09208004 0.05018176 ... 0.         0.11386363 0.12357096]
 [0.26477877 0.39990861 0.28066441 ... 0.11386363 0.         0.35297928]
 [0.1659443  0.41927736 0.8046731  ... 0.12357096 0.35297928 0.        ]]
adjacency [[0.         0.11785122 0.30179518 ... 0.0953715  0.15973888 0.77703644]
 [0.11785122 0.         0.16471074 ... 0.23339387 0.6185102  0.1872688 ]
 [0.30179518 0.16471074 0.         ... 0.32472097 0.43521061 0.29139477]
 ...
 [0.0953715  0.23339387 0.32472097 ... 0.         0.69815066 0.24257109]
 [0.15973888 0.6185102  0.43521061 ... 0.69815066 0.         0.310577  ]
 [0.77703644 0.1872688  0.29139477 ... 0.24257109 0.310577   0.        ]]
adjacency [[0.         0.0894966  0.16786662 ... 0.24918544 0.24769863 0.16956836]
 [0.08949

In [65]:
a = matrix_method.spectral_mutual_information_via_affinity(X,X)
print(f"Mutual info {a}")

adjacency [[0.         0.0894966  0.16786662 ... 0.22603731 0.26477877 0.1659443 ]
 [0.0894966  0.         0.2567415  ... 0.09208004 0.39990861 0.41927736]
 [0.16786662 0.2567415  0.         ... 0.05018176 0.28066441 0.8046731 ]
 ...
 [0.22603731 0.09208004 0.05018176 ... 0.         0.11386363 0.12357096]
 [0.26477877 0.39990861 0.28066441 ... 0.11386363 0.         0.35297928]
 [0.1659443  0.41927736 0.8046731  ... 0.12357096 0.35297928 0.        ]]
adjacency [[0.         0.0894966  0.16786662 ... 0.22603731 0.26477877 0.1659443 ]
 [0.0894966  0.         0.2567415  ... 0.09208004 0.39990861 0.41927736]
 [0.16786662 0.2567415  0.         ... 0.05018176 0.28066441 0.8046731 ]
 ...
 [0.22603731 0.09208004 0.05018176 ... 0.         0.11386363 0.12357096]
 [0.26477877 0.39990861 0.28066441 ... 0.11386363 0.         0.35297928]
 [0.1659443  0.41927736 0.8046731  ... 0.12357096 0.35297928 0.        ]]
adjacency [[0.         0.0894966  0.16786662 ... 0.22603731 0.26477877 0.1659443 ]
 [0.08949

In [ ]:
# investigating the laplacian. How am I getting such big eigenvalues?
